In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = f96d136_x
molecule = {'h2o': 'atmpro', 'co2': 0.0004, 'n2o': 3.2e-07}
band = [3]
atmpro = mls
tsfc = 294


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o h2o band3a
  atmpro = mls
  band = 3a
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 0
  molecule = h2o
  ng_adju = [0, 0]
  ng_refs = [2, 6]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(10, 250), (600, 250)]
  tsfc = 294
  vmax = 620
  vmin = 540
  w_diffuse = [(1.9, 1.7), (1.4, 1.4, 1.4, 1.55, 1.6, 1.66)]
  wgt = [(0.7, 0.7), (0.7, 0.5, 0.5, 0.5, 0.5, 0.5)]
o co2 band3a
  atmpro = mls
  band = 3a
  commitnumber = a06b618
  conc = 0.0004
  dv = 0.001
  klin = 6.375563e-24
  molecule = co2
  ng_adju = [0, 0, 0]
  ng_refs = [3, 2, 4]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (10, 250), (500, 250)]
  tsfc = 294
  vmax = 620
  vmin = 540
  w_diffuse = [(1.6, 1.6, 1.7), (1.75, 1.75), (1.55, 1.55, 1.6, 1.85)]
  wgt = [(0.7, 0.3, 0.7), (0

# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1  0.393184                    1 -1.309473
0.000750        2  0.392150                    2  0.073794
0.001052        3  0.403698                    3  0.255663
0.001476        4  0.414500                    4  0.440254
0.002070        5  0.423268                    5  0.626570
0.002904        6  0.428759                    6  0.811925
0.004074        7  0.430491                    7  0.994420
0.005714        8  0.427059                    8  1.172071
0.008015        9  0.417199                    9  1.342507
0.011243       10  0.399614                   10  1.503682
0.015771       11  0.372704                   11  1.653489
0.022122       12  0.334923                   12  1.788029
0.031031       13  0.284540                   13  1.903769
0.043528       14  0.219733                   14  1.998672
0.061057       15  0.171438                   15  2.099896
0.085645       16  0.194250                   16  2.270896
0.120136       17  0.264847                   17  2.504369
0.168516       18  0.322976                   18  2.719162
0.236378       19  0.372871                   19  2.875944
0.331549       20  0.429965                   20  2.977801
0.465100       21  0.512670                   21  3.064135
0.652400       22  0.652597                   22  3.185444
0.915100       23  0.811154                   23  3.236423
1.283650       24  0.780785                   24  3.016818
1.800600       25  0.612281                   25  2.594348
2.525700       26  0.478381                   26  2.159361
3.542800       27  0.378498                   27  1.779272
4.969550       28  0.300502                   28  1.458378
6.970850       29  0.241145                   29  1.188578
9.778100       30  0.194278                   30  0.949143
13.715850      31  0.152132                   31  0.731364
19.239350      32  0.113647                   32  0.538479
26.987250      33  0.086951                   33  0.381212
37.855300      34  0.070629                   34  0.260664
53.100050      35  0.054172                   35  0.166450
73.887500      36  0.036771                   36  0.096190
97.662500      37  0.026113                   37  0.085215
121.437500     38  0.020571                   38  0.070389
145.212500     39  0.010851                   39  0.052331
168.987500     40  0.000545                   40  0.037653
192.762500     41  0.004399                   41  0.037626
216.537500     42  0.025884                   42  0.052263
240.312500     43  0.051967                   43  0.078709
264.087500     44  0.080338                   44  0.111246
287.862500     45  0.109029                   45  0.143517
311.637500     46  0.127294                   46  0.175415
335.412500     47  0.143269                   47  0.200792
359.187500     48  0.160523                   48  0.216279
382.962500     49  0.178517                   49  0.222106
406.737500     50  0.196831                   50  0.219562
430.512500     51  0.215039                   51  0.210048
454.287500     52  0.230139                   52  0.195828
478.062500     53  0.243821                   53  0.178169
501.837500     54  0.257265                   54  0.158784
525.612500     55  0.270299                   55  0.139797
549.387500     56  0.282760                   56  0.122977
573.162500     57  0.294696                   57  0.109715
596.937500     58  0.314987                   58  0.097873
620.712500     59  0.332144                   59  0.088091
644.487500     60  0.343727                   60  0.079460
668.262500     61  0.349933                   61  0.071460
692.037500     62  0.350810                   62  0.064241
715.812500     63  0.347014                   63  0.058204
739.587500     64  0.339288                   64  0.053983
763.362500     65  0.328463                   65  0.052028
78

# Fluxes by Layer

CRD                           CLIRAD                \
                        flug       fldg      fnetg       flug          fldg   
pressure    level                                                             
0.000000    1     -24.866066   0.000000 -24.866066 -15.541766  4.415713e-08   
0.000624    2     -24.866085   0.000048 -24.866037 -15.541899  3.591684e-05   
0.000876    3     -24.866092   0.000067 -24.866026 -15.541947  8.632169e-05   
0.001229    4     -24.866102   0.000094 -24.866009 -15.542007  1.574788e-04   
0.001723    5     -24.866117   0.000132 -24.865984 -15.542082  2.582920e-04   
0.002417    6     -24.866136   0.000187 -24.865950 -15.542174  4.014314e-04   
0.003391    7     -24.866163   0.000263 -24.865900 -15.542284  6.048979e-04   
0.004757    8     -24.866201   0.000370 -24.865830 -15.542412  8.941769e-04   
0.006672    9     -24.866253   0.000520 -24.865734 -15.542557  1.305167e-03   
0.009359    10    -24.866326   0.000725 -24.865601 -15.542713  1.888116e-03   
0.013128    11    -24.866428   0.001005 -24.865422 -15.542866  2.712748e-03   
0.018415    12    -24.866572   0.001383 -24.865189 -15.542992  3.874623e-03   
0.025830    13    -24.866777   0.001882 -24.864895 -15.543049  5.501993e-03   
0.036232    14    -24.867074   0.002530 -24.864544 -15.542963  7.762490e-03   
0.050823    15    -24.867520   0.003356 -24.864164 -15.542614  1.086803e-02   
0.071291    16    -24.868193   0.004444 -24.863749 -15.541778  1.512406e-02   
0.100000    17    -24.869152   0.006064 -24.863088 -15.540016  2.108593e-02   
0.140271    18    -24.870424   0.008600 -24.861825 -15.536629  2.964672e-02   
0.196760    19    -24.872053   0.012389 -24.859663 -15.530706  4.192050e-02   
0.275997    20    -24.874044   0.017880 -24.856164 -15.520975  5.918677e-02   
0.387100    21    -24.876287   0.025782 -24.850505 -15.505684  8.309051e-02   
0.543100    22    -24.878360   0.037329 -24.841031 -15.482391  1.164265e-01   
0.761700    23    -24.878715   0.054583 -24.824132 -15.447886  1.644158e-01   
1.068500    24    -24.873968   0.079315 -24.794653 -15.400414  2.345764e-01   
1.498800    25    -24.863469   0.108615 -24.754855 -15.344319  3.322714e-01   
2.102400    26    -24.850190   0.139114 -24.711076 -15.285932  4.594012e-01   
2.949000    27    -24.835224   0.172123 -24.663101 -15.228293  6.183381e-01   
4.136600    28    -24.818919   0.209065 -24.609854 -15.173276  8.136535e-01   
5.802500    29    -24.801679   0.251125 -24.550554 -15.123608  1.051808e+00   
8.139200    30    -24.783665   0.299860 -24.483805 -15.082864  1.340095e+00   
11.417000   31    -24.765059   0.356688 -24.408371 -15.057346  1.683148e+00   
16.014700   32    -24.747524   0.422008 -24.325515 -15.056687  2.080853e+00   
22.464000   33    -24.735327   0.496634 -24.238693 -15.093225  2.528814e+00   
31.510500   34    -24.732014   0.586499 -24.145514 -15.180878  3.025023e+00   
44.200100   35    -24.739676   0.700330 -24.039346 -15.340343  3.576353e+00   
62.000000   36    -24.767654   0.842531 -23.925123 -15.610221  4.197232e+00   
85.775000   37    -24.833649   1.012084 -23.821565 -16.038842  4.896781e+00   
109.550000  38    -24.923255   1.175232 -23.748023 -16.549432  5.647389e+00   
133.325000  39    -25.030564   1.340476 -23.690088 -17.127711  6.423926e+00   
157.100000  40    -25.162603   1.503075 -23.659529 -17.754151  7.197762e+00   
180.875000  41    -25.329943   1.671948 -23.657995 -18.403667  7.953332e+00   
204.650000  42    -25.526904   1.881297 -23.645607 -19.052285  8.707927e+00   
228.425000  43    -25.740613   2.167904 -23.572709 -19.700042  9.502888e+00   
252.200000  44    -25.968923   2.542569 -23.426354 -20.348183  1.037272e+01   
275.975000  45    -26.214597   3.014501 -23.200097 -21.002828  1.134070e+01   
299.750000  46    -26.481855   3.588819 -22.893036 -21.673456  1.241556e+01   
323.525000  47    -26.764260   4.229724 -22.534536 -22.343814  1.357999e+01   
347.300000  48    -27.058341   4.927296 -22.131046 -23.005153  1.480689e+01

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')